In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [37]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        self.num_filters_total = num_filters * len(filter_sizes)
        self.W = nn.Embedding(vocab_size, embedding_size) # 모든 단어를 embedding_size 크기로 임베딩
        self.Weight = nn.Linear(self.num_filters_total, num_classes, bias=False)
        self.Bias = nn.Parameter(torch.ones([num_classes]))
        self.filter_list = nn.ModuleList([nn.Conv2d(1, num_filters, (size, embedding_size)) for size in filter_sizes])
        
    def forward(self, X):
        # X: sentences의 각 문장 별로 각 단어를 index로 바꾼 tensor (batch_size, sequence_length)
        embedded_chars = self.W(X) # X의 모든 index를 embedding vector로 바꿈 (batch_size, sequence_length, embedding_size)
        embedded_chars = embedded_chars.unsqueeze(dim=1) # (batch_size, 1, sequence_length, embedding_size) # 차원이 1인 차원을 추가
        
        pooled_outputs = []
        for idx, conv in enumerate(self.filter_list):
            h = F.relu(conv(embedded_chars))
            mp = nn.MaxPool2d((sequence_length - filter_sizes[idx] + 1, 1))
            pooled = mp(h).permute(0, 3, 2, 1)
            pooled_outputs.append(pooled)
        
        h_pool = torch.cat(pooled_outputs, len(filter_sizes))
        h_pool_flat = torch.reshape(h_pool, [-1, self.num_filters_total])
        model = self.Weight(h_pool_flat) + self.Bias
        
        return model

In [38]:
embedding_size = 2
sequence_length = 3
num_classes = 2
filter_sizes = [2, 2, 2]
num_filters = 3

In [39]:
# 각 문장의 단어 수는 3 (= sequence_length)
sentences = ["i love you", "he loves me", "she likes baseball", \
             "i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0] # 1: good, 0: bad

batch_size = len(sentences)

In [40]:
word_list = " ".join(sentences).split()
word_list = list(set(word_list)) # 중복 단어 제거
word_dict = {word: idx for idx, word in enumerate(word_list)}

vocab_size = len(word_list)

In [41]:
word_dict

{'she': 0,
 'that': 1,
 'for': 2,
 'he': 3,
 'i': 4,
 'you': 5,
 'me': 6,
 'sorry': 7,
 'awful': 8,
 'this': 9,
 'baseball': 10,
 'loves': 11,
 'likes': 12,
 'hate': 13,
 'is': 14,
 'love': 15}

In [42]:
model = TextCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [43]:
# 각 문장 별로 단어들의 index를 가져옴
inputs = [ [word_dict[word] for word in sentence.split()] for sentence in sentences ]
inputs = torch.LongTensor(inputs) # List -> Tensor

targets = torch.LongTensor(labels)

In [44]:
# Training

EPOCHS = 5000
for epoch in range(EPOCHS):
    optimizer.zero_grad()
    output = model(inputs)
    
    loss = criterion(output, targets)
    if (epoch + 1) % 1000 == 0:
        print('Epoch: {:04d}  loss: {:.6f}'.format(epoch + 1, loss))
    
    loss.backward()
    optimizer.step()

Epoch: 1000  loss: 0.001323
Epoch: 2000  loss: 0.000246
Epoch: 3000  loss: 0.000088
Epoch: 4000  loss: 0.000039
Epoch: 5000  loss: 0.000020


In [74]:
# Test

test_text = "love is you"
test_inputs = [ [word_dict[word] for word in test_text.split()] ]
test_inputs = torch.LongTensor(test_inputs)

predict = model(test_inputs).data.max(1, keepdim=True)[1]
if predict[0][0] == 0:
    print(test_text, "is Bad mean...")
else:
    print(test_text, "is Good mean...")

love is you is Good mean...


In [28]:
# inputs

tensor([[ 4, 15,  5],
        [ 3, 11,  6],
        [ 0, 12, 10],
        [ 4, 13,  5],
        [ 7,  2,  1],
        [ 9, 14,  8]])

In [32]:
# W = nn.Embedding(vocab_size, embedding_size)
# W(inputs)

tensor([[[ 1.6104, -0.6037],
         [ 1.0356,  0.9875],
         [ 0.5431,  0.9982]],

        [[-0.7609, -0.9334],
         [-1.3904,  1.5549],
         [-2.2182, -1.0644]],

        [[-0.4611, -2.2616],
         [ 1.1070,  0.2645],
         [-0.2258, -0.4881]],

        [[ 1.6104, -0.6037],
         [-0.8689, -0.3644],
         [ 0.5431,  0.9982]],

        [[-0.9141,  0.4710],
         [-1.0834,  1.9768],
         [-0.8339,  0.4260]],

        [[-1.0548, -1.7132],
         [-1.0154, -0.7481],
         [-0.7182,  0.9317]]], grad_fn=<EmbeddingBackward>)